In [ ]:
import pandas as pd

# 업로드한 파일 경로
file_path = "train.csv"

# CSV 파일 불러오기
df = pd.read_csv(file_path)

# 라벨이 1인 데이터만 필터링
df_filtered = df[df['label'] == 1].copy()

# 필요한 열만 구성, category는 'review'로 지정
df_final = pd.DataFrame({
    "text": df_filtered["text"],
    "labels": 1,
    "category": "review"
})

df_final.to_csv("ai_reviews.csv", index=False)

In [ ]:
# 결측치와 중복률 계산
missing_counts = df_final.isnull().sum()
duplicate_count = df_final.duplicated().sum()
total_count = len(df_final)

missing_counts, duplicate_count, total_count

(text        0
 labels      0
 category    0
 dtype: int64,
 np.int64(0),
 5948)

In [ ]:
import pandas as pd

# JSON 파일 불러오기
df = pd.read_json("train.jsonl", lines=True)

# 평점이 1~5점 사이만 필터링
df_filtered = df[df['review_star'].isin([1, 2, 3, 4, 5])].copy()

# 문장이 3문장 이상인 데이터만 필터링 (마침표를 기준으로 문장 수 판단)
df_filtered = df_filtered[df_filtered["en_segment"].str.count(r'[.!?]') >= 2]  # 2개 이상의 마침표 → 3문장 이상 추정

# 총 37687개 균등 샘플링
target_total = 37687
per_score = target_total // 5

# 점수별 균등 샘플링
balanced_samples = pd.concat([
    df_filtered[df_filtered['review_star'] == score].sample(per_score, random_state=42)
    for score in range(1, 6)
])

# 최종 DataFrame 구성
df_final = pd.DataFrame({
    "text": balanced_samples["en_segment"].astype(str),
    "labels": 1,
    "category": "review"
})

# 정보 출력
print("✅ 결측치 수:\n", df_final.isnull().sum())
print("🔁 중복된 행 수:", df_final.duplicated().sum())
print("📊 총 데이터 수:", len(df_final))

# 저장 (선택)
df_final.to_csv("ai_reviews_balanced_3sent.csv", index=False)

✅ 결측치 수:
 text        0
labels      0
category    0
dtype: int64
🔁 중복된 행 수: 0
📊 총 데이터 수: 37685


In [ ]:
# ai_review.csv와 ai_review_plus.csv 파일 불러오기
df_ai_review = pd.read_csv("ai_reviews.csv")
df_ai_review_plus = pd.read_csv("ai_reviews_plus.csv")

# 두 데이터프레임 합치기
df_combined = pd.concat([df_ai_review, df_ai_review_plus], ignore_index=True)

# 합쳐진 데이터프레임의 정보 확인 (선택)
print("✅ 합쳐진 데이터프레임 결측치 수:\n", df_combined.isnull().sum())
print("🔁 합쳐진 데이터프레임 중복된 행 수:", df_combined.duplicated().sum())
print("📊 합쳐진 데이터프레임 총 데이터 수:", len(df_combined))

# 합쳐진 데이터프레임 저장 (선택)
df_combined.to_csv("combined_ai_reviews.csv", index=False)


✅ 합쳐진 데이터프레임 결측치 수:
 text        0
labels      0
category    0
dtype: int64
🔁 합쳐진 데이터프레임 중복된 행 수: 0
📊 합쳐진 데이터프레임 총 데이터 수: 43633


In [ ]:
import pandas as pd

# combined_ai_reviews.csv 파일 불러오기
try:
    df_combined = pd.read_csv("combined_ai_reviews.csv")

    # 결측치 확인 및 개수 계산
    missing_counts = df_combined.isnull().sum()

    # 중복 데이터 확인 및 개수 계산
    duplicate_count = df_combined.duplicated().sum()

    # 결과 출력
    print("✅ combined_ai_reviews.csv 파일의 결측치 수:\n", missing_counts)
    print("🔁 combined_ai_reviews.csv 파일의 중복된 행 수:", duplicate_count)

except FileNotFoundError:
    print("🚨 combined_ai_reviews.csv 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"🚨 파일을 읽거나 처리하는 중 오류가 발생했습니다: {e}")

✅ combined_ai_reviews.csv 파일의 결측치 수:
 text        0
labels      0
category    0
dtype: int64
🔁 combined_ai_reviews.csv 파일의 중복된 행 수: 0
